In [92]:
cd /content

/content


In [93]:
!nvidia-smi

Mon Jul 29 08:38:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    29W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
! rm -rf xlnet_train

In [95]:
! rm -rf xlnet_train
! git clone https://github.com/Stodgers/xlnet_train.git
! pip install sentencepiece

Cloning into 'xlnet_train'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 148 (delta 48), reused 131 (delta 33), pack-reused 0
Receiving objects: 100% (148/148), 1.83 MiB | 4.59 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [96]:
cd /content/xlnet_train/data/sogou_news

/content/xlnet_train/data/sogou_news


In [97]:
! python sogou_data_utils.py

T-news_sohusite_xml.smarty.txt


In [98]:
cd /content/xlnet_train/data/sogou_news

/content/xlnet_train/data/sogou_news


In [99]:
# con = "spm_train \
# 	--input = 233.txt \
# 	--model_prefix = sp10m.cased.v3 \
# 	--vocab_size = 32000 \
# 	--character_coverage = 0.99995 \
# 	--model_type = unigram \
# 	--control_symbols = < cls >，< sep >，< pad >，< mask >，< eod > \
# 	--user_defined_symbols = < eop >，。，（，），“， - ， - ，£，€ \
#  	--shuffle_input_sentence \
#  	--input_sentence_size = 10000000"
# !{con}
import sentencepiece as spm
spm.SentencePieceTrainer.Train("--input=T-news_sohusite_xml.smarty.txt --model_prefix=sp10m.cased.v3 --vocab_size=3000 --character_coverage=0.99995 --model_type=unigram --control_symbols=<cls>,<sep>,<pad>,<mask>,<eod> --user_defined_symbols=<eop>,.,(,),“,”,-,£,€ --shuffle_input_sentence --input_sentence_size=10000")

True

In [0]:
! mv T-news_sohusite_xml.smarty.txt /content/xlnet_train/xlnet
! mv sp10m.cased.v3.model /content/xlnet_train/xlnet
! mv sp10m.cased.v3.vocab /content/xlnet_train/xlnet

In [101]:
cd /content/xlnet_train/xlnet

/content/xlnet_train/xlnet


In [102]:
pip install tensorflow-gpu==1.14

In [103]:
comond = "python data_utils.py \
	--bsz_per_host=8 \
	--num_core_per_host=1 \
	--seq_len=128 \
	--reuse_len=64 \
	--input_glob=T-news_sohusite_xml.smarty.txt \
	--save_dir=tf_info\
	--num_passes=20 \
	--bi_data=False \
	--sp_path=sp10m.cased.v3.model \
	--mask_alpha=6 \
	--mask_beta=1 \
	--num_predict=21"
!{comond}

W0729 08:39:20.215664 140408673802112 deprecation_wrapper.py:119] From data_utils.py:914: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0729 08:39:20.215926 140408673802112 deprecation_wrapper.py:119] From data_utils.py:914: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0729 08:39:20.216053 140408673802112 deprecation_wrapper.py:119] From data_utils.py:915: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0729 08:39:20.216763 140408673802112 deprecation_wrapper.py:119] From data_utils.py:181: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0729 08:39:20.216945 140408673802112 deprecation_wrapper.py:119] From data_utils.py:182: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W0729 08:39:20.217258 140408673802112 deprecation_wrapper.py:119] From data_utils.py:206: The name tf.gfile.Open is d

In [0]:
comond = "python train_gpu.py \
  --record_info_dir=tf_info/tfrecords \
  --train_batch_size=8 \
  --num_core_per_host=1 \
  --seq_len=128 \
  --reuse_len=64 \
  --mem_len=96 \
  --perm_size=64 \
  --n_layer=6 \
  --d_model=1024 \
  --d_embed=1024 \
  --n_head=16 \
  --d_head=64 \
  --d_inner=4096 \
  --untie_r=true \
  --mask_alpha=6 \
  --mask_beta=1 \
  --num_predict=21 \
  --model_dir=ch_model \
  --uncased=true \
  --bi_data=False \
  --save_steps=1000"
!{comond}

W0729 08:39:28.767211 139915755067264 deprecation_wrapper.py:119] From /content/xlnet_train/xlnet/model_utils.py:295: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0729 08:39:28.777159 139915755067264 deprecation_wrapper.py:119] From train_gpu.py:328: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0729 08:39:28.777740 139915755067264 deprecation_wrapper.py:119] From train_gpu.py:315: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0729 08:39:28.777890 139915755067264 deprecation_wrapper.py:119] From train_gpu.py:315: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0729 08:39:28.777999 139915755067264 deprecation_wrapper.py:119] From train_gpu.py:319: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0729 08:39:28.778070 139915755067264 train_gpu.py:319] n_token 32000
W0729 0